In [100]:
from nltk.corpus import stopwords
import pandas as pd
import re
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [101]:
train = pd.read_csv('EI-oc-En-anger-train.txt', sep='\t', error_bad_lines=False)
train.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
dev = pd.read_csv('EI-oc-En-anger-dev.txt', sep='\t', error_bad_lines=False)
dev.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
test= pd.read_csv("EI-oc-En-anger-test.txt", sep='\t', error_bad_lines=False)
test.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]

<ipython-input-101-788454d9d8d3>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv('EI-oc-En-anger-train.txt', sep='\t', error_bad_lines=False)
<ipython-input-101-788454d9d8d3>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dev = pd.read_csv('EI-oc-En-anger-dev.txt', sep='\t', error_bad_lines=False)
<ipython-input-101-788454d9d8d3>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test= pd.read_csv("EI-oc-En-anger-test.txt", sep='\t', error_bad_lines=False)


In [102]:
test

,ID,Tweet,Affect_Dimension,Intensity_Class
0,2018-En-02328,@PageShhh1 I know you mean well but I'm offend...,anger,NONE
1,2018-En-02617,"Let go of resentment, it will hold you back, d...",anger,NONE
2,2018-En-01021,"No, I'm not 'depressed because of the weather,...",anger,NONE
3,2018-En-03737,#AmarnathTerrorAttack Muslims are killing eve...,anger,NONE
4,2018-En-03407,Prepare to suffer the sting of Ghost Rider's p...,anger,NONE
...,...,...,...,...
997,2018-En-00089,That morning when you get half-way to work and...,anger,NONE
998,2018-En-00891,@SenecaReads @zimyix @MouthyBuddha I bet he ne...,anger,NONE
999,2018-En-03356,@markaw214 @appleofeden_3 @jjasq @XxPLWxX Ring...,anger,NONE
1000,2018-En-03011,Have to go to a occupational services place fo...,anger,NONE


In [103]:
train['Intensity_Class'] = pd.Categorical(train['Intensity_Class'])
train['Intensity_Class'] =train.Intensity_Class.cat.codes

dev['Intensity_Class'] = pd.Categorical(dev['Intensity_Class'])
dev['Intensity_Class'] = dev.Intensity_Class.cat.codes

test['Intensity_Class'] = pd.Categorical(test['Intensity_Class'])
test['Intensity_Class'] = test.Intensity_Class.cat.codes

In [104]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stop_words.update(["now","let", 'zero', 'one', 'two', 'three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

#Removing Stop Words

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

train['Tweet'] = train['Tweet'].apply(removeStopWords)
test['Tweet'] = test['Tweet'].apply(removeStopWords)
dev['Tweet'] = dev['Tweet'].apply(removeStopWords)

#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train['Tweet'] = train['Tweet'].apply(stemming)
dev['Tweet'] = dev['Tweet'].apply(stemming)
test['Tweet'] = test['Tweet'].apply(stemming)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
from keras.utils import to_categorical
target=train.Intensity_Class.values
y=to_categorical(target)

target=dev.Intensity_Class.values
c=to_categorical(target)

max_features = 13000
max_words = 50
batch_size = 128
epochs = 3
num_classes=4

X_train = train["Tweet"]
X_val = dev["Tweet"]
Y_train = y
Y_val = c

In [106]:
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,GRU,LSTM,Embedding
from keras.optimizers import Adam
from keras.layers import SpatialDropout1D,Dropout,Bidirectional,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [107]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [108]:
X_test = tokenizer.texts_to_sequences(test['Tweet'])
X_test =pad_sequences(X_test, maxlen=max_words)

In [109]:
X_train =pad_sequences(X_train, maxlen=max_words)
X_val = pad_sequences(X_val, maxlen=max_words)
X_test =pad_sequences(X_test, maxlen=max_words)

In [110]:
model3_LSTM=Sequential()
model3_LSTM.add(Embedding(max_features,100,mask_zero=True))
model3_LSTM.add(LSTM(64,dropout=0.4,return_sequences=True))
model3_LSTM.add(LSTM(32,dropout=0.5,return_sequences=False))
model3_LSTM.add(Dense(num_classes,activation='sigmoid'))
model3_LSTM.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.001),metrics=['accuracy'])
model3_LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1300000   
                                                                 
 lstm_2 (LSTM)               (None, None, 64)          42240     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 4)                 132       
                                                                 
Total params: 1,354,788
Trainable params: 1,354,788
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [111]:
history3=model3_LSTM.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
14/14 [==============================] - 23s 594ms/step - loss: 0.6778 - accuracy: 0.2887 - val_loss: 0.6537 - val_accuracy: 0.3093
Epoch 2/3
14/14 [==============================] - 3s 210ms/step - loss: 0.5989 - accuracy: 0.3510 - val_loss: 0.5925 - val_accuracy: 0.3041
Epoch 3/3
14/14 [==============================] - 3s 220ms/step - loss: 0.5723 - accuracy: 0.3133 - val_loss: 0.5877 - val_accuracy: 0.2990


In [117]:
#y_pred3=model3_LSTM.predict_classes(X_test, verbose=1)
import numpy as np
predict_x=model3_LSTM.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

classes_x

32/32 [==============================] - 1s 35ms/step


array([2, 2, 2, ..., 2, 2, 0])